In [2]:
import quartz # type: ignore
import time

gate_set = ['h', 'cz', 'rz', 'rx1', 'rx3', 'x', 'add']
ecc_file_path: str = '../ecc_set/rigetti_5_ecc.json'
no_increase = False
include_nop = True
quartz_context = quartz.QuartzContext(
    gate_set=gate_set,
    filename=ecc_file_path,
    no_increase=no_increase,
    include_nop=include_nop,
)
print(f'{quartz_context.num_xfers}')
def qasm_to_graph(qasm_str: str) -> quartz.PyGraph:
    graph = quartz.PyGraph.from_qasm_str(context=quartz_context, qasm_str=qasm_str)
    return graph

def is_nop(xfer_id: int) -> bool:
    return quartz_context.get_xfer_from_id(id=xfer_id).is_nop

qasm_file = '../rigetti_circs/twolocalrandom_nativegates_rigetti_qiskit_opt0_10_norm.qasm'
with open(qasm_file) as f:
    qasm = f.read()
s_time = time.time_ns()
graph = qasm_to_graph(qasm)
print(graph.gate_count)

2189
here 1
1105


In [1]:
import quartz # type: ignore
import time

gate_set = ['h', 'cx', 'x', 'rz', 'add']
ecc_file_path: str = '../ecc_set/nam_ecc.json'
no_increase = False
include_nop = True

quartz_context = quartz.QuartzContext(
    gate_set=gate_set,
    filename=ecc_file_path,
    no_increase=no_increase,
    include_nop=include_nop,
)
print(f'{quartz_context.num_xfers}')

def qasm_to_graph(qasm_str: str) -> quartz.PyGraph:
    graph = quartz.PyGraph.from_qasm_str(context=quartz_context, qasm_str=qasm_str)
    return graph

def is_nop(xfer_id: int) -> bool:
    return quartz_context.get_xfer_from_id(id=xfer_id).is_nop

/home/jinjun/miniconda3/envs/quartz/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


8669


In [3]:
# qasm_file = '../outputs/2022-06-26/08-45-11/barenco_tof_3/38_3/24_38_2_26_8622.qasm'
qasm_file = '../nam_circs/gf2^7_mult.qasm'
with open(qasm_file) as f:
    qasm = f.read()
s_time = time.time_ns()
graph = qasm_to_graph(qasm)
e_time = time.time_ns()
graph2 = qasm_to_graph(qasm)
print(f'dur: {(e_time - s_time) / 1e6} ms')
print(graph.gate_count)
print(hash(graph))

s_time = time.time_ns()
qasm_str = graph.to_qasm_str()
e_time = time.time_ns()
print(f'dur: {(e_time - s_time) / 1e6} ms')

graph_again = qasm_to_graph(qasm_str)
hash(graph_again) == hash(graph)

dur: 8.205839 ms
669
-8556416482083842346
dur: 2.883815 ms


True

In [11]:
import dgl
with open('../../nam_circs/adder_8.qasm') as f:
    qasm = f.read()
pg1 = qasm_to_graph(qasm)
g1 = pg1.to_dgl_graph()

In [13]:
print(f'{g1.ndata["gate_type"].shape}')
subg, new_indices = dgl.khop_out_subgraph(g1, 400, k=6)
print(f'{subg}\n{new_indices}\n{subg.ndata["gate_type"].shape}')
print(f'{new_indices.shape}')


torch.Size([900])
Graph(num_nodes=43, num_edges=112,
      ndata_schemes={'gate_type': Scheme(shape=(), dtype=torch.int32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'src_idx': Scheme(shape=(), dtype=torch.int32), 'dst_idx': Scheme(shape=(), dtype=torch.int32), 'reversed': Scheme(shape=(), dtype=torch.int32), '_ID': Scheme(shape=(), dtype=torch.int64)})
tensor([22])
torch.Size([43])
torch.Size([1])


In [5]:
import torch
import dgl
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling

with open('../../nam_circs/adder_8.qasm') as f:
    qasm = f.read()
pg1 = qasm_to_graph(qasm)
with open('../../nam_circs/barenco_tof_3.qasm') as f:
    qasm = f.read()
pg2 = qasm_to_graph(qasm)

g1 = pg1.to_dgl_graph()
g2 = pg2.to_dgl_graph()

bg = dgl.batch([g1, g2])
print(bg.ndata['gate_type'].shape)

pool = SumPooling()

h = torch.rand(958, 32)
pooled_h = pool(bg, h)
print(pooled_h.shape)

torch.Size([958])
torch.Size([2, 32])


In [34]:
import torch
import dgl
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling

with open('../../nam_circs/adder_8.qasm') as f:
    qasm = f.read()
pg1 = qasm_to_graph(qasm)
with open('../../nam_circs/barenco_tof_3.qasm') as f:
    qasm = f.read()
pg2 = qasm_to_graph(qasm)

g1 = pg1.to_dgl_graph()
g2 = pg2.to_dgl_graph()
# g1 = g2

In [35]:
# print(g1.edata)
g1.edata['w'] = torch.cat([
    torch.unsqueeze(g1.edata['src_idx'], 1),
    torch.unsqueeze(g1.edata['dst_idx'], 1),
    torch.unsqueeze(g1.edata['reversed'], 1),
], dim=1)
# print(g1.edata)
# print(g1.edata['src_idx'].shape)
# print(g1.edata['w'].shape)
g13 = dgl.remove_self_loop(g1)
# print(g13.edata['w'].shape)
# print(g13.edata['src_idx'], g13.edata['dst_idx'], g13.edata['src_idx'].shape)
# print(g1.num_edges(), g13.num_edges())
# print(g1.edges())
print(g1.edges()[0].shape, '\n', g1.edges())
print(g13.edges()[0].shape, '\n', g13.edges())
print(g13.edata)

torch.Size([2570]) 
 (tensor([  0,   0,   1,  ..., 897, 898, 899]), tensor([  8,  84,   9,  ..., 894, 895, 897]))
torch.Size([2570]) 
 (tensor([  0,   0,   1,  ..., 897, 898, 899]), tensor([  8,  84,   9,  ..., 894, 895, 897]))
{'src_idx': tensor([0, 1, 0,  ..., 0, 0, 0], dtype=torch.int32), 'dst_idx': tensor([0, 0, 0,  ..., 0, 0, 1], dtype=torch.int32), 'reversed': tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.int32), 'w': tensor([[0, 0, 0],
        [1, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 1]], dtype=torch.int32)}


In [37]:
g12 = dgl.add_self_loop(g1)
print(g12.edata)
print(g12.edata['w'].shape, g12.edata['src_idx'].shape)

{'src_idx': tensor([0, 1, 0,  ..., 0, 0, 0], dtype=torch.int32), 'dst_idx': tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32), 'reversed': tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32), 'w': tensor([[0, 0, 0],
        [1, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)}
torch.Size([3470, 3]) torch.Size([3470])


In [16]:
i = 2570
print(g12.edata['src_idx'][i], g12.edata['dst_idx'][i])

print(g1.ndata['gate_type'].shape, g12.ndata['gate_type'])

tensor(0, dtype=torch.int32) tensor(0, dtype=torch.int32)
torch.Size([900]) tensor([6, 6, 0, 6, 0, 0, 6, 0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6,
        5, 5, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6,
        6, 5, 6, 5, 6, 5, 6, 5, 5, 0, 5, 0, 5, 0, 5, 0, 6, 6, 6, 6, 6, 6, 6, 6,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 6,
        6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 6, 5, 5, 0, 5, 5, 0,
        5, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 0, 6, 6, 6, 6, 5, 5, 5, 5, 6, 6,
        6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5,
        6, 5, 5, 5, 5, 5, 0, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 0, 6, 6, 6,
        6, 6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5,
        6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 6, 5, 5, 0, 5, 0, 5, 0, 5, 0, 6, 6, 6, 6,
        6, 6, 6, 5, 5, 0, 1, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 5, 5, 5, 6, 6, 6,
        5, 5, 5, 6, 6, 6, 5, 6, 5, 6, 5, 6, 

In [3]:
print(graph.gate_count)
print(graph.cx_count)

58
24


In [8]:
bg.batch_num_nodes()
# x0 = torch.zeros(0)
# x0 += torch.rand(3, 16)

tensor([900,  58])

In [6]:
av_xfers = graph.available_xfers_parallel(
    context=quartz_context, node=graph.get_node_from_id(id=22))
print(av_xfers)

[98, 100, 188, 189, 499, 501, 507, 511, 8668]


In [14]:
import natsort
from natsort import natsorted
graphs = []
graph_gc = []
graph_cc = []
record_dir = 'barenco_tof_3'
path_dirs = natsorted(os.listdir(record_dir))
for path_dir in path_dirs:
    path_dir_full = os.path.join(record_dir, path_dir)
    cir_file = natsorted(os.listdir(path_dir_full))[-1]
    cir_file_path = os.path.join(path_dir_full, cir_file)
    with open(cir_file_path) as f:
        qasm = f.read()
    graph = qasm_to_graph(qasm)
    graphs.append(graph)
    graph_gc.append(graph.gate_count)
    graph_cc.append(graph.cx_count)
print(graphs)
print(graph_gc)
print(graph_cc)


[<quartz.core.PyGraph object at 0x7f93ffe5e700>, <quartz.core.PyGraph object at 0x7f94001c8d40>, <quartz.core.PyGraph object at 0x7f93fffd6900>, <quartz.core.PyGraph object at 0x7f9400b30c00>, <quartz.core.PyGraph object at 0x7f93ffd05b80>, <quartz.core.PyGraph object at 0x7f93ffd071c0>, <quartz.core.PyGraph object at 0x7f93ffd06140>, <quartz.core.PyGraph object at 0x7f93ffd04800>, <quartz.core.PyGraph object at 0x7f93ffd04140>, <quartz.core.PyGraph object at 0x7f93ffd05580>, <quartz.core.PyGraph object at 0x7f93ffd07600>, <quartz.core.PyGraph object at 0x7f93ffd06840>, <quartz.core.PyGraph object at 0x7f93ffd06900>, <quartz.core.PyGraph object at 0x7f93ffd05980>, <quartz.core.PyGraph object at 0x7f93ffd04840>, <quartz.core.PyGraph object at 0x7f93ffe93fc0>, <quartz.core.PyGraph object at 0x7f93ffe93f80>, <quartz.core.PyGraph object at 0x7f93ffe937c0>]
[36, 36, 36, 37, 38, 39, 40, 41, 43, 45, 46, 48, 49, 50, 52, 54, 56, 56]
[14, 14, 14, 15, 16, 17, 18, 19, 21, 21, 20, 22, 23, 24, 24, 2

In [12]:
sorted(os.listdir('barenco_tof_3/36_1'))

['0_37_0_0_0.qasm',
 '10_39_1_31_2569.qasm',
 '11_39_0_32_8344.qasm',
 '12_38_1_30_2602.qasm',
 '13_39_-1_15_6877.qasm',
 '14_38_1_15_6898.qasm',
 '15_38_0_29_97.qasm',
 '16_38_0_16_102.qasm',
 '17_38_0_29_102.qasm',
 '18_38_0_16_103.qasm',
 '19_39_-1_30_4259.qasm',
 '1_37_0_15_103.qasm',
 '20_38_1_30_2618.qasm',
 '21_39_-1_30_2665.qasm',
 '22_39_0_34_189.qasm',
 '23_37_2_33_8646.qasm',
 '24_38_-1_11_2665.qasm',
 '25_37_1_11_2618.qasm',
 '26_37_0_16_102.qasm',
 '27_37_0_16_97.qasm',
 '28_38_-1_15_6894.qasm',
 '29_40_-2_14_94.qasm',
 '2_38_-1_11_4259.qasm',
 '30_40_0_13_8335.qasm',
 '31_40_0_13_8308.qasm',
 '32_39_1_11_2618.qasm',
 '33_38_1_16_1888.qasm',
 '34_36_2_15_8622.qasm',
 '3_37_1_11_2618.qasm',
 '4_38_-1_15_6894.qasm',
 '5_37_1_15_6878.qasm',
 '6_39_-2_15_2041.qasm',
 '7_37_2_15_2042.qasm',
 '8_39_-2_33_94.qasm',
 '9_40_-1_31_2568.qasm']